In [1]:
import numpy as np
import pandas as pd

# 지하철역 - 행정동위치 전처리

In [2]:
#지하철역 좌표 데이터 불러오기
지하철역 = pd.read_csv('서울시 역사마스터 정보.csv',encoding='cp949')
지하철역

,역사_ID,역사명,호선,위도,경도
0,9996,미사,5호선,127.193877,37.560927
1,9995,강일,5호선,127.175930,37.557490
2,4929,김포공항,김포골드라인,126.801868,37.562360
3,4928,고촌,김포골드라인,126.770345,37.601243
4,4927,풍무,김포골드라인,126.732387,37.612488
...,...,...,...,...,...
733,154,종로5가,1호선,127.001849,37.570926
734,153,종로3가,1호선,126.991847,37.570406
735,152,종각,1호선,126.982923,37.570161
736,151,시청,1호선,126.977088,37.565715


In [3]:
#좌표 -> 행정동으로 변환 
import pandas as pd
import requests 
import sys
import json
import datetime 



def json_request(url='', encoding='utf-8', success=None, error=lambda e: print('%s : %s' % (e, datetime.now()), file=sys.stderr)):
    headers = {'Authorization': 'KakaoAK {}'.format(APP_KEY)}
    resp = requests.get(url, headers=headers)
    # print('%s : success for request [%s]' % (datetime.now(), url))
    return resp.text


def reverse_geocode(longitude, latitude):
    # 파라미터 최적화하여 url 생성
    url = '%s?x=%s&y=%s' %(URL, longitude, latitude)
    # json request
    try:
        # print('try')
        json_req = json_request(url=url)
        json_data = json.loads(json_req)
        json_doc = json_data.get('documents')[1]
        json_name = json_doc.get('region_3depth_name')
        json_code = json_doc.get('code')
    except:
        # print('nan')
        json_name = 'NaN'
        json_code = 'NaN'
    return json_name, json_code

 
def get_address(x,y):
    address = []        
    json_name, json_code = reverse_geocode(x, y) 
    address.append(json_name)
    return address # 전처리 함수에서 주소 리스트 받아서 데이터프레임에 추가


def get_code(x,y):
    code = []        
    json_name, json_code = reverse_geocode(x, y) 
    code.append(json_code)
    return code # 전처리 함수에서 행정구역코드 리스트 받아서 데이터프레임에 추가
    


APP_KEY = 'dc125ad9de42a4ee620f0bdb2e623811' 
URL = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json'

df_all = 지하철역
df = df_all[['역사명','호선', '경도', '위도']] # 기관ID, 병원경도, 병원위도 열만 추출 (도로명주소는 일부러 날림)


for i in range(len(df)): 
    x_crd = float(df.loc[i, ['위도']])
    y_crd = float(df.loc[i, ['경도']])
    address = get_address(x_crd, y_crd)
    code = get_code(x_crd, y_crd)
    df.loc[i, ['행정동명']] = address
    df.loc[i, ['행정동_코드']] = code
    
    if i%100 == 0:
        print('{} 번째 경위도 좌표계에 대한 정보 불러오는 중...'.format(i))
    

0 번째 경위도 좌표계에 대한 정보 불러오는 중...
100 번째 경위도 좌표계에 대한 정보 불러오는 중...
200 번째 경위도 좌표계에 대한 정보 불러오는 중...
300 번째 경위도 좌표계에 대한 정보 불러오는 중...
400 번째 경위도 좌표계에 대한 정보 불러오는 중...
500 번째 경위도 좌표계에 대한 정보 불러오는 중...
600 번째 경위도 좌표계에 대한 정보 불러오는 중...
700 번째 경위도 좌표계에 대한 정보 불러오는 중...


In [5]:
train = df
#읍면동 코드 제거
for i in range(len(train)):
    train['행정동_코드'][i]=train['행정동_코드'][i][0:8]
train

<ipython-input-5-eed868ee4777>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['행정동_코드'][i]=train['행정동_코드'][i][0:8]


,역사명,호선,경도,위도,행정동명,행정동_코드
0,미사,5호선,37.560927,127.193877,미사1동,41450610
1,강일,5호선,37.557490,127.175930,상일2동,11740526
2,김포공항,김포골드라인,37.562360,126.801868,방화2동,11500640
3,고촌,김포골드라인,37.601243,126.770345,고촌읍,41570253
4,풍무,김포골드라인,37.612488,126.732387,사우동,41570540
...,...,...,...,...,...,...
733,종로5가,1호선,37.570926,127.001849,종로5.6가동,11110630
734,종로3가,1호선,37.570406,126.991847,종로1.2.3.4가동,11110615
735,종각,1호선,37.570161,126.982923,종로1.2.3.4가동,11110615
736,시청,1호선,37.565715,126.977088,소공동,11140520


In [6]:
#필요한 열만 추출 + 데이터 저장
train=train.filter(['역사명','호선','행정동명','행정동_코드'])
train.to_csv("지하철역_행정동위치.csv",mode="w",encoding='euc-kr',index=False)
train

,역사명,호선,행정동명,행정동_코드
0,미사,5호선,미사1동,41450610
1,강일,5호선,상일2동,11740526
2,김포공항,김포골드라인,방화2동,11500640
3,고촌,김포골드라인,고촌읍,41570253
4,풍무,김포골드라인,사우동,41570540
...,...,...,...,...
733,종로5가,1호선,종로5.6가동,11110630
734,종로3가,1호선,종로1.2.3.4가동,11110615
735,종각,1호선,종로1.2.3.4가동,11110615
736,시청,1호선,소공동,11140520


# 행정동별 - 역사수 전처리

In [13]:
#행정동별로 역사수 정리 코드
train_counts=train.groupby(['행정동명','행정동_코드'], as_index=False).count()
train_counts=train_counts.filter(['행정동명','행정동_코드','역사명'])
train_counts.rename(columns={'역사명':'역사수'},inplace=True)
train_counts

,행정동명,행정동_코드,역사수
0,가능동,41150595,1
1,가락1동,11710631,2
2,가락2동,11710632,2
3,가락본동,11710620,1
4,가산동,11545510,3
...,...,...,...
472,회현동,11140540,4
473,효자동,41281560,2
474,효창동,11170580,1
475,휘경1동,11230720,1


## 승하차 인원 -> 호선, 역명 으로 묶어서 7월 승하차인원수 합치기

In [14]:
#승하차인원 데이터 불러오기
승하차=pd.read_csv('CARD_SUBWAY_MONTH_202107.csv')
승하차

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
20210701,중앙선,상봉(시외버스터미널),6102,6098,20210704,NaN
20210701,중앙선,망우,7706,7633,20210704,NaN
20210701,중앙선,양원,1987,2052,20210704,NaN
20210701,중앙선,구리,13576,13748,20210704,NaN
20210701,중앙선,도농,10145,9146,20210704,NaN
...,...,...,...,...,...,...
20210731,경원선,청량리(서울시립대입구),11320,13138,20210803,NaN
20210731,경원선,외대앞,4261,4279,20210803,NaN
20210731,경원선,신이문,4666,4313,20210803,NaN
20210731,2호선,용두(동대문구청),1292,1364,20210803,NaN


In [15]:
#데이터 열명 정리 
승하차.rename(columns={'사용일자':'호선','노선명':'역명','역명':'승차총승객수','승차총승객수':'하차총승객수','하차총승객수':'사용_일자','등록일자':'사용일자'},inplace=True)
승하차=승하차.rename_axis('등록일자').reset_index()
승하차=승하차.filter(['호선','역명','승차총승객수','하차총승객수'])
승하차

,호선,역명,승차총승객수,하차총승객수
0,중앙선,상봉(시외버스터미널),6102,6098
1,중앙선,망우,7706,7633
2,중앙선,양원,1987,2052
3,중앙선,구리,13576,13748
4,중앙선,도농,10145,9146
...,...,...,...,...
18627,경원선,청량리(서울시립대입구),11320,13138
18628,경원선,외대앞,4261,4279
18629,경원선,신이문,4666,4313
18630,2호선,용두(동대문구청),1292,1364


In [16]:
#승하차승객수 합치기
승하차_merged= 승하차.groupby(['호선','역명'],as_index=False).sum()
승하차_merged

,호선,역명,승차총승객수,하차총승객수
0,1호선,동대문,290903,275373
1,1호선,동묘앞,224920,231201
2,1호선,서울역,969833,934082
3,1호선,시청,475729,475120
4,1호선,신설동,338087,331583
...,...,...,...,...
602,중앙선,원덕,9372,9191
603,중앙선,중랑,142833,136184
604,중앙선,지평,2016,2207
605,중앙선,팔당,27522,27617


In [17]:
#데이터상 오류 -> 어떻게 해결? 평균대치법 or 제거
승하차_merged[승하차_merged['하차총승객수']<100]

,호선,역명,승차총승객수,하차총승객수
91,3호선,충무로,23,0
197,6호선,신내,8,0
202,6호선,연신내,28,0
397,경원선,창동,5,0
403,경의선,계양,2,0
408,경의선,김포공항,1,0
504,분당선,복정,13,0
578,일산선,지축,5,0


In [18]:
#역사명, 호선을 통해 train 행정동 데이터와 승하차승객수 데이터 합치기
승하차인원 = pd.merge(left=train,right=승하차_merged,how='inner',left_on=['역사명','호선'],right_on=['역명','호선'])
승하차인원

,역사명,호선,행정동명,행정동_코드,역명,승차총승객수,하차총승객수
0,미사,5호선,미사1동,41450610,미사,363386,363577
1,미사,5호선,미사1동,41450610,미사,363386,363577
2,강일,5호선,상일2동,11740526,강일,78648,79295
3,강일,5호선,상일2동,11740526,강일,78648,79295
4,신설동,우이신설선,용신동,11230536,신설동,52426,49719
...,...,...,...,...,...,...,...
541,종로5가,1호선,종로5.6가동,11110630,종로5가,534042,529232
542,종로3가,1호선,종로1.2.3.4가동,11110615,종로3가,605684,564592
543,종각,1호선,종로1.2.3.4가동,11110615,종각,765082,743351
544,시청,1호선,소공동,11140520,시청,475729,475120


In [19]:
#(역사명-호선)중복제거
승하차인원=승하차인원.drop_duplicates(['역사명','호선'], keep='first')
#필요한 열만 뽑기
승하차인원=승하차인원.filter(['행정동명','행정동_코드','승차총승객수','하차총승객수'])
#행정동명,행정동코드별 승하차수 합 구하기
승하차인원=승하차인원.groupby(['행정동명','행정동_코드'], as_index=False).sum()

In [20]:
#하루평균 승하차인원
승하차인원['승차총승객수']=round(승하차인원['승차총승객수']/31.0,2)
승하차인원['하차총승객수']=round(승하차인원['하차총승객수']/31.0,2)

승하차인원

,행정동명,행정동_코드,승차총승객수,하차총승객수
0,가능동,41150595,5344.32,5155.87
1,가락1동,11710631,11786.71,12440.65
2,가락2동,11710632,10720.84,10966.13
3,가락본동,11710620,2918.03,2832.87
4,가산동,11545510,60587.00,63269.10
...,...,...,...,...
368,회현동,11140540,48929.90,48658.45
369,효자동,41281560,3295.87,3200.10
370,효창동,11170580,5325.90,4802.42
371,휘경1동,11230720,18534.90,17871.29


# 승하차인원 + 역사수 데이터 합치기

In [30]:
#승하차인원 + 역사수 합치기
승하차인원_역사수 = pd.merge(left=train_counts,right=승하차인원,how='inner',on=['행정동명','행정동_코드'])
승하차인원_역사수

,행정동명,행정동_코드,역사수,승차총승객수,하차총승객수
0,가능동,41150595,1,5344.32,5155.87
1,가락1동,11710631,2,11786.71,12440.65
2,가락2동,11710632,2,10720.84,10966.13
3,가락본동,11710620,1,2918.03,2832.87
4,가산동,11545510,3,60587.00,63269.10
...,...,...,...,...,...
368,회현동,11140540,4,48929.90,48658.45
369,효자동,41281560,2,3295.87,3200.10
370,효창동,11170580,1,5325.90,4802.42
371,휘경1동,11230720,1,18534.90,17871.29


In [31]:
#승하차인원 역사별로 보기위해 역사수로 나눔
for i in range(len(승하차인원_역사수)):
    승하차인원_역사수['승차총승객수'][i] = 승하차인원_역사수['승차총승객수'][i]/ 승하차인원_역사수['역사수'][i]
    승하차인원_역사수['하차총승객수'][i] = 승하차인원_역사수['하차총승객수'][i]/ 승하차인원_역사수['역사수'][i]
#열이름 수정
승하차인원_역사수.rename(columns={'승차총승객수':'역사별_승차승객수','하차총승객수':'역사별_하차승객수'},inplace=True)
승하차인원_역사수

<ipython-input-31-48ca2fb4f810>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  승하차인원_역사수['승차총승객수'][i] = 승하차인원_역사수['승차총승객수'][i]/ 승하차인원_역사수['역사수'][i]
<ipython-input-31-48ca2fb4f810>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  승하차인원_역사수['하차총승객수'][i] = 승하차인원_역사수['하차총승객수'][i]/ 승하차인원_역사수['역사수'][i]


,행정동명,행정동_코드,역사수,역사별_승차승객수,역사별_하차승객수
0,가능동,41150595,1,5344.320000,5155.8700
1,가락1동,11710631,2,5893.355000,6220.3250
2,가락2동,11710632,2,5360.420000,5483.0650
3,가락본동,11710620,1,2918.030000,2832.8700
4,가산동,11545510,3,20195.666667,21089.7000
...,...,...,...,...,...
368,회현동,11140540,4,12232.475000,12164.6125
369,효자동,41281560,2,1647.935000,1600.0500
370,효창동,11170580,1,5325.900000,4802.4200
371,휘경1동,11230720,1,18534.900000,17871.2900


In [29]:
#내보내기
승하차인원_역사수.to_csv("행정동별_지하철역_역사수_승하차인원.csv",mode="w",encoding='euc-kr',index=False)